<a href="https://colab.research.google.com/github/gonzaq94/NER-LLM-classification/blob/main/fine_tune_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
COMET_API_KEY = "70hdKpWAXsGBl956kbVxML85N"

In [30]:
import comet_ml
import comet_llm
import os
from datasets import load_dataset
import random
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


## Load dataset

Take 1000 random examples of the CONLL dataset.

In [5]:
# Load the CONLL dataset
dataset_train = load_dataset("conll2003", split="train").shuffle().select(range(1000))
dataset_valid = load_dataset("conll2003", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [6]:
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 1000
})
Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 3250
})


In [7]:
for i in range(5):
    print(dataset_train[i])

{'id': '11812', 'tokens': ['The', 'individual', 'Names', ',', 'however', ',', 'now', 'must', 'decide', 'whether', 'to', 'accept', 'Lloyd', "'s", 'settlement', 'offer', 'or', 'reject', 'the', 'offer', 'and', 'pursue', 'litigation', '.'], 'pos_tags': [12, 16, 24, 6, 30, 6, 30, 20, 37, 15, 35, 37, 22, 27, 21, 21, 10, 37, 12, 21, 10, 37, 21, 7], 'chunk_tags': [11, 12, 12, 0, 3, 0, 3, 21, 22, 17, 21, 22, 11, 11, 12, 12, 0, 21, 11, 12, 0, 21, 11, 0], 'ner_tags': [0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': '4686', 'tokens': ['3.', 'Margaret', 'Crowley', '(', 'Australia', ')', '2:02.40'], 'pos_tags': [22, 22, 22, 4, 22, 5, 11], 'chunk_tags': [11, 12, 12, 0, 11, 0, 11], 'ner_tags': [0, 1, 2, 0, 5, 0, 0]}
{'id': '5513', 'tokens': ['Stabaek', '20', '7', '8', '5', '41', '34', '29'], 'pos_tags': [21, 11, 11, 11, 11, 11, 11, 11], 'chunk_tags': [11, 12, 12, 12, 12, 12, 12, 12], 'ner_tags': [3, 0, 0, 0, 0, 0, 0, 0]}
{'id': '2014', 'tokens': ['"', 'I', 'got', 'more'

In [8]:
print(dataset_train['tokens'])
print(dataset_train['ner_tags'])

[['The', 'individual', 'Names', ',', 'however', ',', 'now', 'must', 'decide', 'whether', 'to', 'accept', 'Lloyd', "'s", 'settlement', 'offer', 'or', 'reject', 'the', 'offer', 'and', 'pursue', 'litigation', '.'], ['3.', 'Margaret', 'Crowley', '(', 'Australia', ')', '2:02.40'], ['Stabaek', '20', '7', '8', '5', '41', '34', '29'], ['"', 'I', 'got', 'more', 'aggressive', 'in', 'the', 'second', 'and', 'third', 'sets', 'and', 'the', 'wind', 'picked', 'up', 'and', 'that', 'also', 'affected', 'things', 'because', 'Cedric', 'definitely', 'went', 'off', 'a', 'little', 'bit', '.', '"'], ['67', '69'], ['GOLF', '-', 'SCORES', 'AT', 'WORLD', 'SERIES', 'OF', 'GOLF', '.'], ['Results', 'from', 'the'], ['declared', '(', 'G.', 'Gooch', '111', ',', 'R.', 'Irani', '91', ',', 'P.', 'Prichard', '88', ',', 'D.', 'Robinson', '72', ';'], ['NAC', 'Breda', '1', '(', 'Abdellaoui', '20th', 'penalty', ')', 'NEC', 'Nijmegen', '1', '(', 'Graef', '36th', ')', '.'], ['SEOUL', '1996-08-23'], ['Chalmers', '(', 'Australia',

 ## Load BERT model

Train a BERT on this dataset to classify tokens (the ner_tags column). Report performance.

In [9]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=9)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Dataset preparation

In [23]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        tokens = []
        tokens_labels = []
        attention_masks = []

        for word, label in zip(text, labels):
          # Tokenize each word
          word_encoding = self.tokenizer(
              word,
              add_special_tokens=False,
              max_length=self.max_length // len(text),
              truncation=True,
              return_tensors="pt"
          )

          tokens.extend(word_encoding['input_ids'][0].tolist())
          attention_masks.extend(word_encoding['attention_mask'][0].tolist())
          tokens_labels.extend([label] * len(word_encoding['input_ids'][0]))

        # add padding
        padding_len = self.max_length - len(tokens)
        tokens += [0] * padding_len
        attention_masks += [0] * padding_len
        tokens_labels += [-100] * padding_len

        return {
            'input_ids': torch.tensor(tokens).flatten(),
            'attention_mask': torch.tensor(attention_masks).flatten(),
            'labels': torch.tensor(tokens_labels).flatten()
        }


In [24]:
train_custom_dataset = CustomDataset(
    texts=dataset_train['tokens'],
    labels=dataset_train['ner_tags'],
    tokenizer=tokenizer,
    max_length=128
)

valid_custom_dataset = CustomDataset(
    texts=dataset_valid['tokens'],
    labels=dataset_valid['ner_tags'],
    tokenizer=tokenizer,
    max_length=128
)

## Fine-tune model

In [15]:
# log results and assets to Comet
os.environ["COMET_LOG_ASSETS"] = "True"
os.environ["COMET_WORKSPACE"] = "gonzaq94"
os.environ["COMET_PROJECT_NAME"] = "ner_classif-bert"


In [27]:
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(-1)  # Get the predicted class indices
    true_predictions = [p for (p, l) in zip(predictions.flatten(), labels.flatten()) if l != -100]
    true_labels = [l for l in labels.flatten() if l != -100]

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, true_predictions, average='weighted')
    acc = accuracy_score(true_labels, true_predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [28]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_custom_dataset,
    eval_dataset=valid_custom_dataset,
    compute_metrics=compute_metrics
)


In [17]:
trainer.train()

COMET WARNING: Ending the running experiment and creating a new Experiment because:
workspace doesn't match ('gonzaq94' != '')
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, tensorflow, sklearn, keras.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gonzaq94/ner-classif-bert/87f5f59182ba47798b0403b9a270d4ae

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Step,Training Loss
10,0.250300
20,0.205700
30,0.218600
40,0.217000
50,0.220500
60,0.197300
70,0.184500
80,0.176600
90,0.187600
100,0.174900


TrainOutput(global_step=375, training_loss=0.1459063172340393, metrics={'train_runtime': 90.6526, 'train_samples_per_second': 33.093, 'train_steps_per_second': 4.137, 'total_flos': 195984969984000.0, 'train_loss': 0.1459063172340393, 'epoch': 3.0})

 ## Evaluate the model

In [31]:
eval_results = trainer.evaluate()
print(eval_results)

COMET INFO: An experiment with the same configuration options is already running and will be reused.


{'eval_loss': 0.18180817365646362, 'eval_model_preparation_time': 0.0067, 'eval_accuracy': 0.9568815890974544, 'eval_f1': 0.9574783774856235, 'eval_precision': 0.9588803908595286, 'eval_recall': 0.9568815890974544, 'eval_runtime': 38.7269, 'eval_samples_per_second': 83.921, 'eval_steps_per_second': 5.268}
